# MMVC Trainer

ver.2022/12/19

「Google Colaboratory」を利用してMMVCで利用するVITSの学習を行います。

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。 { display-mode: "form" }

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer" #@param {type:"string"}

%cd $directory
!ls -1

​

In [ ]:
#@title ## 3 GPUの確認
#@markdown GPUの確認を行います。  
#@markdown 割り当てられたGPUのメモリーを確認し、それに合わせてconfigファイルの"batch_size"を指定してください。  
!nvidia-smi

​

In [ ]:
#@title ## 4 ライブラリのインストール
#@markdown 時間がかかります。気長にお待ちください。
!apt-get install espeak
!pip install -r requirements.txt

%cd monotonic_align/
!python setup.py build_ext --inplace
%cd ../

​

In [ ]:
#@title ## 5 tensorboardの起動
#@markdown とりあえず実行しておいてください。
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs --port 5000

​

In [ ]:
#@title ## 6 学習を実行する
#@markdown 学習を実行します。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown ### Settings
#@markdown New / Resume

#@markdown 新規に学習を開始する場合は、Newを選択し、**以下の-c, -m, -fg, -fdを全て設定してください。**

#@markdown 学習を再開する場合は、Resumeを選択し、以下の-c, -mを設定してください。-mで指定したディレクトリの最新のモデルから学習を再開します。 

#@markdown **注意**

#@markdown **学習を再開する前に、"1. Google Driveをマウント"から"5. tensorboardの起動"までを実行してください。ノートブックを開くたびに、毎回必要です。**

New_or_Resume = "New" #@param ["New", "Resume"]

#@markdown -c：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。
config_pass = "configs/train_config.json" #@param {type:"string"}

#@markdown -m：modelの保存先ディレクトリ  
#@markdown **ディレクトリ名を直接指名してください。** 
model_save_dic = "20220306_24000" #@param {type:"string"}

#@markdown -fg：(新規学習時のみ) Fine tuningのベースとなるG_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。
fine_model_g = "fine_model/G_180000.pth" #@param {type:"string"}

#@markdown -fd：(新規学習時のみ) Fine tuningのベースとなるD_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。
fine_model_d = "fine_model/D_180000.pth" #@param {type:"string"}

if New_or_Resume == "New":
  !python train_ms.py -c $config_pass -m $model_save_dic -fg $fine_model_g -fd $fine_model_d
elif New_or_Resume == "Resume":
  !python train_ms.py -c $config_pass -m $model_save_dic

In [ ]:
#@title ## 7 学習結果をONNXファイルに変換

#@markdown 学習結果をONNXファイルに変換します。

#@markdown modelの保存先ディレクトリ  

#@markdown **ディレクトリ名を直接指名してください。** 
model_dir = "20220306_24000" #@param {type:"string"}
!python onnx_export.py --config_file logs/$model_dir/config.json --convert_pth logs/$model_dir/G_latest_99999999.pth

#@markdown 変換が終わるとlogs内にあるmodelの保存先ディレクトリにONNXファイルが生成されています。  

#@markdown クライアントの実行には、このディレクトリ内の **config.json** と **G_latest_99999999.onnx** の2ファイルが必要です。  

#@markdown クライアントのlogsディレクトリ内に、わかりやすいように同じ名前でフォルダを作成し、この2ファイルをコピーしてください。
